In [1]:
from netCDF4 import Dataset
from scipy.io import netcdf #### <--- This is the library to import.
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import cmocean.cm as cmo
import cmaps
import numpy as np
import cmocean

import datetime 
import os

%matplotlib inline
 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

plt.rcParams["font.size"] = 12


In [2]:
def read_netcdf_xr(fid):
    ds = xr.open_mfdataset(fid, combine='by_coords')
    return ds

def plot_200_mask(axes, mask, bathy):
    for ax in axes:
        mask.plot.contour(ax=ax, x='nav_lon', y='nav_lat', linewidths=0.1, colors='k')
        bathy.Bathymetry.plot.contour(ax=ax, x='nav_lon', y='nav_lat', levels=[200], linewidths=1.1, colors='k')
    return

In [ ]:
root_dir = '/home/gkara/Documents/for-tide_paper/'
_f = ['BISCAY-T60_1h_CURL_20110928-20110928.nc', 'BISCAY-T60_1h_CURL_20120224-20120224.nc']
#
curl_ref = read_netcdf_xr([root_dir + 'CURL_ref/' + f for f in _f])
curl_ref = curl_ref.set_coords(['nav_lon', 'nav_lat'])
#
curl_exp1 = read_netcdf_xr([root_dir + 'CURL_exp1/' + f for f in _f])
curl_exp1 = curl_exp1.set_coords(['nav_lon', 'nav_lat'])
#
bathy = read_netcdf_xr(root_dir + 'bathy_meter.nc')
bathy = bathy.set_coords(['nav_lon', 'nav_lat'])

vmin = -.6e-04
vmax = .6e-04
levels = np.linspace(vmin,vmax,49, endpoint=True)

# make plot
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,10))
fig.subplots_adjust(wspace=0.08, hspace=.14)
#
mask = np.isnan(curl_ref.socurl[0,:,:])
im0 = curl_ref.sel(time_counter='2011-09-28T15:30:00.000000000').socurl.plot.contourf(ax=axes[0,0], x='nav_lon', y='nav_lat', levels=levels, vmin=vmin, vmax=vmax, cmap=cmocean.cm.balance, add_colorbar=False)
im1 = curl_exp1.sel(time_counter='2011-09-28T15:30:00.000000000').socurl.plot.contourf(ax=axes[0,1], x='nav_lon', y='nav_lat', levels=levels, vmin=vmin, vmax=vmax, cmap=cmocean.cm.balance, add_colorbar=False)
plot_200_mask(axes=[axes[0,0], axes[0,1]], mask=mask, bathy=bathy)
#
p0 = axes[0,0].get_position().get_points().flatten()
p1 = axes[0,1].get_position().get_points().flatten()
ax_cbar = fig.add_axes([p0[0]+.04, 0.035, p1[2]-p0[0]-.08, 0.025])
cb = plt.colorbar(im0, cax=ax_cbar, orientation='horizontal', extend='both', label='Relative_Vorticity $\zeta$')
cb.set_ticks(np.arange(-.6e-04,1e-03,.1e-04))
cb.formatter.set_powerlimits((0,0))
cb.update_ticks()
#
im2 = curl_ref.sel(time_counter='2012-02-24T15:30:00.000000000').socurl.plot.contourf(ax=axes[1,0], x='nav_lon', y='nav_lat', levels=levels, vmin=vmin, vmax=vmax, cmap=cmocean.cm.balance, add_colorbar=False)
im3 = curl_exp1.sel(time_counter='2012-02-24T15:30:00.000000000').socurl.plot.contourf(ax=axes[1,1], x='nav_lon', y='nav_lat', levels=levels, vmin=vmin, vmax=vmax, cmap=cmocean.cm.balance, add_colorbar=False)
plot_200_mask(axes=[axes[1,0], axes[1,1]], mask=mask, bathy=bathy)
#
axes[0,0].set_xlabel(None); axes[0,1].set_xlabel(None)
axes[0,0].set_xticklabels([]); axes[0,1].set_xticklabels([])
axes[0,1].set_ylabel(None); axes[1,1].set_ylabel(None)
axes[0,1].set_yticklabels([]); axes[1,1].set_yticklabels([])
#
axes[0,0].set_title("(a) TON: 2011-09-28"); axes[0,1].set_title("(b) TOFF: 2011-09-28")
axes[1,0].set_title("(c) TON: 2012-02-24"); axes[1,1].set_title("(d) TOFF: 2012-02-24")

#save figure
fig.savefig(root_dir + '_plots/fig_vorticity.png',dpi=300,bbox_inches='tight')

## Animations

In [ ]:
root_dir = '/home/gkara/Documents/for-tide_paper/'
curl_ref = read_netcdf_xr(root_dir + 'CURL_ref/BISCAY-T60_1h_CURL*')
curl_ref = curl_ref.set_coords(['nav_lon', 'nav_lat'])
#
curl_exp1 = read_netcdf_xr(root_dir + 'CURL_exp1/BISCAY-T60_1h_CURL*')
curl_exp1 = curl_exp1.set_coords(['nav_lon', 'nav_lat'])

vmin = -.5e-04
vmax = .5e-04
levels = np.linspace(vmin,vmax,41, endpoint=True)

for n in range(1):
    print('***' + str(n) + '***')
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(13.5,6))
    fig.subplots_adjust(wspace=0.08)

    im0 = curl_ref.socurl[n,::].plot.contourf(ax=axes[0], x='nav_lon', y='nav_lat', levels=levels, vmin=vmin, vmax=vmax, cmap=cmocean.cm.balance, add_colorbar=False)
    im1 = curl_exp1.socurl[n,::].plot.contourf(ax=axes[1], x='nav_lon', y='nav_lat', levels=levels, vmin=vmin, vmax=vmax, cmap=cmocean.cm.balance, add_colorbar=False)
    mask = np.isnan(curl_ref.socurl[n,:,:])
    mask.plot.contour(ax=axes[0], x='nav_lon', y='nav_lat', linewidths=0.2,colors='black')
    mask.plot.contour(ax=axes[1], x='nav_lon', y='nav_lat', linewidths=0.2,colors='black')

    axes[1].yaxis.tick_right()
    axes[0].yaxis.set_ticks_position('both')
    axes[1].yaxis.set_ticks_position('both')
    axes[1].set_ylabel(None)

    p0 = axes[0].get_position().get_points().flatten()
    p1 = axes[1].get_position().get_points().flatten()
    ax_cbar = fig.add_axes([p0[0]+.04, 0, p1[2]-p0[0]-.08, 0.025])
    cb = plt.colorbar(im0, cax=ax_cbar, orientation='horizontal', extend='both', label='Relative_Vorticity $\zeta$')
    cb.set_ticks(np.arange(-.5e-04,1e-03,.1e-04))
    cb.formatter.set_powerlimits((0,0))
    cb.update_ticks()
    
    #save figure
#     fig.savefig(root_dir + '_plots/curl%04d.png'%n,dpi=300,bbox_inches='tight')
#     plt.close()